In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import warnings
import dill
import pickle as pkl

import mrtool

sys.path.append('../')
import paths_vsn6 as pth
import mr_functions
import plotting_functions

import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
output_dir = pth.CURR_VSN_DIR
df = pd.read_csv(pth.CLEANED_REG_DF_WITH_SENS)

In [5]:
df = pd.concat([df.loc[df['RatioID'] == i].iloc[[0]] for i in df['RatioID'].unique()], axis=0)

In [7]:
response_name = 'log_icer'
se_name = 'log_icer_se'
spline_cov = 'log_GDP_2017usd_per_cap'
study_id_name = 'ArticleID'
data_id_name = 'RatioID'

In [8]:
cwalk_params = pd.read_csv(pth.CWALK_PARAM_SUMMARY).set_index('covariate')
cwalk_params

cwalk_covs = ['log_total_cost_per_cap', 'DiscountRate','CostsDiscountRate', 'coverage','pentavalent',
              'payer','efficacy'] # 'ltd_and_societal',



In [9]:
df = df[(df[response_name].notnull()) &\
        (~np.isinf(df[response_name])) &\
        (df[spline_cov].notnull()) &\
        (df['log_burden_variable'].notnull())]
df = df[df[cwalk_covs].notnull().all(axis=1)]

In [12]:
cov_dict = {spline_cov: df[spline_cov].to_numpy(),
            'log_burden_variable': df['log_burden_variable'].to_numpy()}
cov_dict.update({w: df[w].to_numpy() for w in cwalk_covs})

cwalk_priors = cwalk_params[['beta', 'se_beta']]
cwalk_param_dict = {w: cwalk_priors.loc[w, ['beta', 'se_beta']].to_numpy() for w in cwalk_covs}
cwalk_prior_dict = {key: value for key, value in cwalk_param_dict.items()}

In [ ]:
if not os.path.exists(pth.SIGNAL_MR_PKL):
    signal_mr = mr_functions.fit_signal_model(df,
                                              resp_name=response_name, se_name=se_name, spline_cov=spline_cov,
                                              study_id_name=study_id_name, data_id_name=data_id_name,
                                              other_cov_names=cwalk_covs + ['log_burden_variable'],
                                              other_cov_gpriors=cwalk_param_dict,
                                              h=0.1, num_samples=20, deg=2, n_i_knots=2, knots_type='domain',
                                              knot_bounds = np.array([[0.1, 0.6], [0.4, 0.9]]),
                                              interval_sizes = np.array([[0.1, 0.7], [0.1, 0.7], [0.1, 0.7]]))
    with open(pth.SIGNAL_MR_PKL, 'wb') as out_file:
        dill.dump(signal_mr, out_file)
else:
    print('signal mr object has already been fitted')
    with open(pth.SIGNAL_MR_PKL, 'rb') as in_file:
        signal_mr = dill.load(in_file)

In [15]:
signal_df = mr_functions.create_signal(signal_mr, spline_cov, spline_cov_values=df[spline_cov].to_numpy(),
                                       data_id_name=data_id_name, data_ids=df[data_id_name].to_numpy())
w_df = mr_functions.get_ws(signal_mr,
                 data_id_name=data_id_name)
signal_df = signal_df.merge(w_df, on=[data_id_name])

df = df.merge(signal_df[[data_id_name, 'new_spline_cov', 'w']],
              on=[data_id_name])

In [16]:
full_df = df.copy()
df = df[df['w'] > 0.5].copy().reset_index(drop=True)

In [ ]:
if not os.path.exists(pth.SEL_COVS_PKL):
    selected_covs = mr_functions.select_covariates(df=df,
                                                   candidate_covs=['log_burden_variable',
                                                                   'not_lifetime',
                                                                   'qalys'
                                                                  ],
                                                   include_covs=['intercept', 'both_types','new_spline_cov'] + cwalk_covs,
                                                   resp_name=response_name,
                                                   se_name=se_name, study_id_name=study_id_name,
                                                   beta_gprior=cwalk_param_dict
                                                  )

    with open(pth.SEL_COVS_PKL, 'wb') as out_file:
        pkl.dump(selected_covs, out_file, protocol=pkl.HIGHEST_PROTOCOL)
else:
    print('covariates have already been selected.')
    with open(pth.SEL_COVS_PKL, 'rb') as in_file:
        selected_covs = pkl.load(in_file)

In [19]:
df['null_study_id'] = df['RatioID']
if not os.path.exists(pth.CV_RESULTS):
    cv_sds, cv_mses = mr_functions.k_fold_cv_gaussian_prior(k=10,
                                                            df=df,
                                                            resp_name=response_name,
                                                            se_name=se_name,
                                                            study_id_name=study_id_name,
                                                            data_id_name=data_id_name,
                                                            covs=selected_covs,
                                                            beta_gpriors=cwalk_prior_dict,
                                                            initial_upper_prior_sd=1.0,
                                                            num_sds_per_step=5)
    cv_sds = cv_sds[np.argsort(cv_mses)]
    cv_mses = cv_mses[np.argsort(cv_mses)]
    
    cv_results = pd.DataFrame({'sd': cv_sds, 'mse': cv_mses})
    cv_results.to_csv(pth.CV_RESULTS)
else:
    cv_results = pd.read_csv(pth.CV_RESULTS)
    cv_mses = cv_results['mse'].to_numpy()
    cv_sds = cv_results['sd'].to_numpy()

In [ ]:
upper_bound = np.quantile(np.log(cv_mses), 0.55)
lower_bound = np.log(cv_mses)[0]
upper_bound = upper_bound + (upper_bound - lower_bound) * 1.1
lower_bound = lower_bound - (upper_bound - lower_bound) * 1.1

msk = np.log(cv_mses) < upper_bound
xvals = cv_sds[msk]
yvals = np.log(cv_mses)[msk]

left_bound = np.min(xvals)
right_bound = np.max(xvals)
max_minus_min = right_bound - left_bound
left_bound = left_bound - max_minus_min * 1.1
right_bound = right_bound + max_minus_min * 1.1

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14, 8))
ax.scatter(x=xvals, y=yvals)
ax.set_ylim((lower_bound, upper_bound))
ax.set_xlim((left_bound, right_bound))

In [22]:
prior_sd = cv_results['sd'].to_numpy()[np.argmin(cv_results['mse'].to_numpy())]

gpriors = {v: np.array([0, prior_sd / df[v].std()])
           for v in selected_covs if v not in ['intercept'] + list(cwalk_prior_dict.keys())}

gpriors.update(cwalk_prior_dict)
gpriors.update({'intercept': [np.array([0, np.inf]),
                              np.array([0, np.inf])]})

In [23]:
#FINAL_MODEL_PKL = pth.CURR_VSN_DIR + 'final_model.pkl'
if not os.path.exists(pth.FINAL_MODEL_PKL):
    mr = mr_functions.fit_with_covs(df=df, covs=selected_covs,
                                    resp_name=response_name, se_name=se_name,
                                    study_id_name=study_id_name,
                                    data_id_name=data_id_name,
                                    z_covs=['intercept'],
                                    trim_prop=0.0, spline_cov=None,
                                    gprior_dict=gpriors,
                                    inner_max_iter=2000, outer_max_iter=1000)

    with open(pth.FINAL_MODEL_PKL, 'wb') as out_file:
        dill.dump(mr, out_file)
else:
    with open(pth.FINAL_MODEL_PKL, 'rb') as in_file:
        mr = dill.load(in_file)

In [24]:
np.random.seed(5032198)
beta_samples = mrtool.core.other_sampling.sample_simple_lme_beta(1000, mr)

mr_summary = mr_functions.summarize_parameters(mr, 'log_GDP_per_cap')

mr_summary[['beta', 'beta_se', 'beta_variance', 'gamma']] = np.round(mr_summary[['beta', 'beta_se', 'beta_variance', 'gamma']],
                                                                     decimals=4)

mr_summary.to_csv(pth.FINAL_MODEL_PARAM_SUMMARY)


beta_samples_pd = pd.DataFrame(beta_samples)
beta_samples_pd.to_csv('file_path/beta_samples.csv')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14, 8))
ax.scatter(df['log_GDP_2017usd_per_cap'], df['new_spline_cov'])
plt.savefig(pth.SPLINE_TRANSFORM_PLOT)

In [ ]:
plotting_functions.visualize_spline(
    signal_mr,
    'log_GDP_2017usd_per_cap',
    df['log_GDP_2017usd_per_cap'].to_numpy(),
    x_on_log_scale=True,
    out_file_name=output_dir + 'spline_transformation_plot_11_15_21.pdf',
    x_label='GDP per capita (USD)')

In [28]:
new_spline_cov = mr_functions.create_signal(signal_mr=signal_mr,
                                            spline_cov=spline_cov,
                                            spline_cov_values=df[spline_cov].to_numpy(),
                                            data_id_name=data_id_name,
                                            data_ids=df[data_id_name].to_numpy())

In [29]:
# Write out df with new spline cov for predictions/cost-saving logistic regressions
df.to_csv('file_path/df_with_new_spline_cov.csv')

In [30]:
df = df[df[[v for v in selected_covs + ['log_icer']]].notnull().all(axis=1)]
df = df[~np.isinf(df['log_icer'])]

In [31]:
fit_df = mr_functions.create_fit_df(mr=mr, df=df,
                                    resp_name=response_name,
                                    study_id_name=study_id_name,
                                    other_id_col_names=[], data_id_name=data_id_name)

r2s = mr_functions.r2(mr, fit_df, response_name)

In [32]:
fit_df.to_csv('file_path/fit_df.csv')

In [32]:
r2s

,R_squared,RMSE,Sample_size
fitted_fe_and_re,0.961194,0.606611,NaN
fitted_fe_only,0.943804,0.737565,1211.0


In [33]:
## R^2 for only the base-case analyses
df = df[df['reference_scenario'] == "1"]

In [35]:
fit_df = mr_functions.create_fit_df(mr=mr, df=df,
                                    resp_name=response_name,
                                    study_id_name=study_id_name,
                                    other_id_col_names=[], data_id_name=data_id_name)

In [36]:
r2_base_cases = mr_functions.r2(mr, fit_df, response_name)

In [37]:
r2_base_cases = r2_base_cases.rename({'Sample_size': 'Sample_size_base_cases',
                                      'R_squared': 'R_squared_base_cases',
                                      'RMSE': 'RMSE_base_cases'},
                                     axis=1)
r2s = r2s.join(r2_base_cases)

In [38]:
r2s

,R_squared,RMSE,Sample_size,R_squared_base_cases,RMSE_base_cases,Sample_size_base_cases
fitted_fe_and_re,0.961194,0.606611,NaN,0.953357,0.593155,NaN
fitted_fe_only,0.943804,0.737565,1211.0,0.929415,0.748240,349.0


In [39]:
r2s.loc['fitted_fe_only', 'Sample_size_base_cases'] = fit_df.shape[0]
if not os.path.exists(pth.R2s):
    r2s.to_csv(pth.R2s)

In [41]:
# Predictions at 90% universal coverage
preds_df = pd.read_csv(pth.CLEANED_PREDS_DF)
preds_df['log_vaccine_cost_2017usd'] = np.log(preds_df['vaccine_cost_2017usd'])

In [42]:
preds_df['idx'] = np.arange(preds_df.shape[0])

preds_df = mr_functions.create_predictions(mr, signal_mr, preds_df,
                                           response_name, se_name, selected_covs,
                                           study_id_name, data_id_name,
                                           beta_samples=beta_samples,
                                           seed=8721)
preds_df = preds_df.reset_index()
preds_df.to_csv(pth.PREDICTIONS, index=False)

In [43]:
# Predictions at DTP3 coverage estimate
preds_df_dtp3 = pd.read_csv(pth.CLEANED_PREDS_DF)
preds_df_dtp3 = preds_df_dtp3.drop('coverage',axis=1)
preds_df_dtp3['log_vaccine_cost_2017usd'] = np.log(preds_df_dtp3['vaccine_cost_2017usd'])
preds_df_dtp3['coverage'] = preds_df_dtp3['dtp3_coverage']

In [44]:
preds_df_dtp3['idx'] = np.arange(preds_df_dtp3.shape[0])

preds_df_dtp3 = mr_functions.create_predictions(mr, signal_mr, preds_df_dtp3,
                                           response_name, se_name, selected_covs,
                                           study_id_name, data_id_name,
                                           beta_samples=beta_samples,
                                           seed=8721)
preds_df_dtp3 = preds_df_dtp3.reset_index()
preds_df_dtp3.to_csv(pth.PREDICTIONS_DTP3, index=False)

In [ ]:
# Predictions at Rotavirus coverage estimate
preds_df_rota = pd.read_csv(pth.CLEANED_PREDS_DF)
preds_df_rota = preds_df_rota.drop('coverage',axis=1)
preds_df_rota['log_vaccine_cost_2017usd'] = np.log(preds_df_rota['vaccine_cost_2017usd'])
preds_df_rota['coverage'] = preds_df_rota['mean_rota']

In [ ]:
preds_df_rota['idx'] = np.arange(preds_df_rota.shape[0])

preds_df_rota = mr_functions.create_predictions(mr, signal_mr, preds_df_rota,
                                           response_name, se_name, selected_covs,
                                           study_id_name, data_id_name,
                                           beta_samples=beta_samples,
                                           seed=8721)
preds_df_rota = preds_df_rota.reset_index()
preds_df_rota.to_csv(pth.PREDICTIONS_ROTA, index=False)

In [ ]:
outlier_df = full_df[['ArticleID', 'RatioID', 'w']]
orig_df = pd.read_csv(pth.CLEANED_REG_DF_WITH_SENS)
outlier_df = outlier_df.merge(orig_df[['ArticleID', 'RatioID',
                                       'log_GDP_2017usd_per_cap', 'log_icer', 'log_vaccine_cost_2017usd', 'log_burden_variable']],
                              on=['ArticleID', 'RatioID'], how='left'
                             )

plotting_functions.plot_quartiles_with_ui(mr,
                                          y_axis_var='log_icer',
                                          x_axis_var='log_vaccine_cost_2017usd',
                                          data_id_name='RatioID',
                                          group_var='log_burden_variable',
                                          spline_transform_df=None, spline_var=None,
                                          beta_samples=beta_samples,
                                          plot_title=None,#'ICER vs. vaccine cost by HPV DALYs per capita quartile',
                                          group_var_name_display='Rotavirus DALYs per capita',
                                          y_axis_var_display='ICER (2017 US$/DALY Averted)',
                                          x_axis_var_display='Vaccine Cost (2017 US$)',
                                          outliers=outlier_df.loc[outlier_df['w']< 0.5],                                           
                                         )
plotting_functions.plot_quartiles_with_ui(mr,
                                          y_axis_var='log_icer',
                                          x_axis_var='log_burden_variable',
                                          data_id_name='RatioID',
                                          group_var='log_vaccine_cost_2017usd',
                                          spline_transform_df=None, spline_var=None,
                                          beta_samples=beta_samples,
                                          plot_title=None,#'ICER vs. HPV DALYs per capita by vaccine cost quartile',
                                          group_var_name_display='Vaccine Cost',
                                          y_axis_var_display='ICER (2017 US$/DALY Averted)',
                                          x_axis_var_display='Rotavirus DALYs per 100,000 population',
                                          outliers=outlier_df.loc[outlier_df['w']< 0.5],
                                          x_decimals=-1, x_scale=1e5,
                                         )
plotting_functions.plot_quartiles_with_ui(mr,
                                          y_axis_var='log_icer',
                                          x_axis_var='log_GDP_2017usd_per_cap',
                                          data_id_name='RatioID',
                                          group_var='log_vaccine_cost',
                                          spline_transform_df = df[['RatioID',
                                                                    'log_GDP_2017usd_per_cap',
                                                                    'new_spline_cov']],
                                          spline_var='new_spline_cov',
                                          beta_samples=beta_samples,
                                          plot_title=None,#'ICER vs. GDP per capita by vaccine cost quartile',
                                          group_var_name_display='Vaccine cost (2017 US$)',
                                          y_axis_var_display='ICER (2017 US$/DALY Averted)',
                                          x_axis_var_display='GDP per capita (2017 US$)',
                                          outliers=outlier_df.loc[outlier_df['w']< 0.5],
                                         )
plotting_functions.plot_quartiles_with_ui(mr,
                                          y_axis_var='log_icer',
                                          x_axis_var='log_vaccine_cost_2017usd',
                                          data_id_name='RatioID',
                                          group_var='log_GDP_2017usd_per_cap',
                                          spline_transform_df = df[['RatioID',
                                                                    'log_GDP_2017usd_per_cap',
                                                                    'new_spline_cov'
                                                                   ]],
                                          spline_var='new_spline_cov',
                                          beta_samples=beta_samples,
                                          plot_title=None,#'ICER vs. vaccine cost by GDP per capita quartile',
                                          x_axis_var_display='Vaccine cost (2017 US$)',
                                          y_axis_var_display='ICER (2017 US$/DALY Averted)',
                                          group_var_name_display='GDP per capita (2017 US$)',
                                          outliers=outlier_df.loc[outlier_df['w']< 0.5],
                                         )
plotting_functions.plot_quartiles_with_ui(mr,
                                          y_axis_var='log_icer',
                                          x_axis_var='log_burden_variable',
                                          data_id_name='RatioID',
                                          group_var='log_GDP_2017usd_per_cap',
                                          spline_transform_df = df[['RatioID',
                                                                    'log_GDP_2017usd_per_cap',
                                                                    'new_spline_cov'
                                                                   ]],
                                          spline_var='new_spline_cov',
                                          beta_samples=beta_samples,
                                          group_var_name_display='GDP per capita',
                                          plot_title=None,
                                          y_axis_var_display='ICER (2017 US$/DALY Averted)',
                                          x_axis_var_display='Rotavirus DALYs per 100,000 population',
                                          x_decimals=-1, x_scale=1e5,
                                          outliers=outlier_df.loc[outlier_df['w']< 0.5],
                                         )

fitteds = mr.predict(mr.data, predict_for_study=False)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14, 8))
ax.scatter(x=fitteds, y=mr.data.obs - fitteds)
plt.savefig(output_dir + 'fits_vs_obs_plot.png')

fitteds = mr.predict(mr.data, predict_for_study=True)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14, 8))
ax.scatter(x=fitteds, y=mr.data.obs - fitteds)
plt.savefig(output_dir + 'fits_vs_resids_plot.png')